In [8]:
import glob
import os
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, Input, MaxPooling2D
from tensorflow.keras.models import load_model

from visualization import SvgGoBoard
from train_utils import ParseForPolicy, ParseForPolicy, CreateDataset

tf.enable_eager_execution()

BOARD_SIZE = 19
NUM_CHANNELS = 7
BATCH_SIZE = 64
SHUFFLE_BUFFER = 12800

In [6]:
FEATURES = {"next": tf.FixedLenFeature([], tf.int64),
            "outcome": tf.FixedLenFeature([], tf.float32),
            "note": tf.VarLenFeature(tf.string),
            "orig": tf.VarLenFeature(tf.float32),
            "b1": tf.VarLenFeature(tf.float32),
            "b2": tf.VarLenFeature(tf.float32),
            "b3": tf.VarLenFeature(tf.float32),
            "w1": tf.VarLenFeature(tf.float32),
            "w2": tf.VarLenFeature(tf.float32),
            "w3": tf.VarLenFeature(tf.float32)}


def _parse_full_example(example):
    return tf.parse_single_example(example, FEATURES)


def _reshape_plane(t):
    return tf.reshape(tf.sparse.to_dense(t), [BOARD_SIZE, BOARD_SIZE])


def _reshape_planes(feature_dict, names):
    return [_reshape_plane(feature_dict[name]) for name in names]


def ParseForPolicyValue(example):
    parsed = _parse_full_example(example)
    planes = _reshape_planes(parsed, ["orig", "b1", "b2", "b3", "w1", "w2", "w3"])
    return tf.stack(planes, axis=2), (parsed["next"], parsed["outcome"])


def BuildCommonLayers(input):
    x = Conv2D(filters=64, kernel_size=(7,7), activation="relu", padding="same",
               data_format='channels_last')(input)
    x = Conv2D(filters=64, kernel_size=(7,7), activation="relu", padding="same",
               data_format='channels_last')(x)
    x = Conv2D(filters=32, kernel_size=(5,5), activation="relu", padding="same",
               data_format='channels_last')(x)
    x = Conv2D(filters=32, kernel_size=(5,5), activation="relu", padding="same",
               data_format='channels_last')(x)
    x = Dropout(0.5)(x)
    x = Conv2D(filters=32, kernel_size=(5,5), activation="relu", padding="same",
               data_format='channels_last')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(512)(x)
    return x


def BuildDualNetModel():
    input = Input(shape=(BOARD_SIZE, BOARD_SIZE, NUM_CHANNELS), name="go_input")
    common = BuildCommonLayers(input)

    # policy network
    x = Dense(BOARD_SIZE*BOARD_SIZE)(common)
    policy_output = Activation("softmax", name="policy_output")(x)

    # value network
    x = Dense(1)(common)
    value_output = Activation("sigmoid", name="value_output")(x)

    model = keras.models.Model(inputs=input, outputs=[policy_output, value_output])
    return model


def CompileModel(model):
    losses = {"policy_output": "sparse_categorical_crossentropy",
              "value_output": "binary_crossentropy"}
    loss_weights = {"policy_output": 1.0, "value_output": 1.0}
    model.compile(optimizer="sgd", loss=losses, loss_weights=loss_weights,
                  metrics=["accuracy"])
    print(model.summary())
    return model

In [7]:
model = BuildDualNetModel()
model = CompileModel(model)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


ResourceExhaustedError: OOM when allocating tensor with shape[11552,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add] name: dense/kernel/Initializer/random_uniform/

In [ ]:
train_set = CreateDataset(glob.glob("/home/tc/SGF/mini/rio/training/data-0000.rio"), BATCH_SIZE)
train_set = train_set.shuffle(buffer_size=SHUFFLE_BUFFER)
model.fit(train_set, epochs=10, steps_per_epoch=1000)